In [1]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Read and Decode BBCI Data with Start-Stop-Markers

This tutorial shows how to read and decode BBCI data with start and stop markers. The data loading part is more complicated and it is advised to read the other tutorials before.

## Setup logging to see outputs

In [2]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()


## Load and preprocess data

This is a bit more complicated than before since we have to add breaks etc. Here I now opt to add breaks do all preprocessings per run and only later combine them together.

In [3]:
import numpy as np
from braindecode.datautil.splitters import concatenate_sets
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne, add_breaks
from braindecode.datasets.bbci import load_bbci_sets_from_folder
from copy import deepcopy
from braindecode.mne_ext.signalproc import resample_cnt, mne_apply
from braindecode.datautil.signalproc import lowpass_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

def create_cnts(folder, runs,):
    # Load data
    cnts = load_bbci_sets_from_folder(folder, runs)
    
    # Now do some preprocessings:
    # Resampling to 250 Hz, lowpass below 38, eponential standardization
    
    new_cnts = []
    for cnt in cnts:
        # Only take some channels 
        #cnt = cnt.drop_channels(['STI 014']) # This would remove stimulus channel
        cnt = cnt.pick_channels(['C3', 'CPz', 'C4'])
        log.info("Preprocessing....")
        cnt = mne_apply(lambda a: lowpass_cnt(a, 38,cnt.info['sfreq'], axis=1), cnt)
        cnt = resample_cnt(cnt, 250)
        # mne apply will apply the function to the data (a 2d-numpy-array)
        # have to transpose data back and forth, since
        # exponential_running_standardize expects time x chans order
        # while mne object has chans x time order
        cnt = mne_apply(lambda a: exponential_running_standardize(
            a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
            cnt)
        new_cnts.append(cnt)
    return new_cnts

In [4]:
from collections import OrderedDict

train_runs = [1,2,3]
train_cnts = create_cnts('/home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R01-8/', 
                         train_runs,)

name_to_start_code = OrderedDict([('Right Hand', 1), ('Feet', 4),
            ('Rotation', 8), ('Words', [10])])

name_to_stop_code = OrderedDict([('Right Hand', [20,21,22,23,24,28,30]),
        ('Feet', [20,21,22,23,24,28,30]),
        ('Rotation', [20,21,22,23,24,28,30]), 
        ('Words', [20,21,22,23,24,28,30])])


2017-10-18 14:50:15,027 INFO : Loading /home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R01-8/AnLaNBD1S001R01_1-1_250Hz.BBCI.mat
Creating RawArray with float64 data, n_channels=64, n_times=151350
    Range : 0 ... 151349 =      0.000 ...   605.396 secs
Ready.
2017-10-18 14:50:15,890 INFO : Loading /home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R01-8/AnLaNBD1S001R02_1-1_250Hz.BBCI.mat
Creating RawArray with float64 data, n_channels=64, n_times=153500
    Range : 0 ... 153499 =      0.000 ...   613.996 secs
Ready.
2017-10-18 14:50:16,737 INFO : Loading /home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R01-8/AnLaNBD1S001R03_1-1_250Hz.BBCI.mat
Creating RawArray with float64 data, n_channels=64, n_times=180700
    Range : 0 ... 180699 =      0.000 ...   722.796 secs
Ready.
2017-10-18 14:50:17,974 INFO : Preprocessing....
2017-10-18 14:50:17,986 INFO : Just copying data, no resampling, since new sampling rate same.
2017-10-18 14:50:18,225 INFO : Preprocessing....
2017-10-18 14:50:18,236 INFO : Just copyin

In [5]:
test_runs = [9,10]
test_cnts = create_cnts('/home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R09-10/', test_runs,)

2017-10-18 14:50:18,588 INFO : Loading /home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R09-10/AnLaNBD1S001R09_1-1_250Hz.BBCI.mat
Creating RawArray with float64 data, n_channels=64, n_times=152050
    Range : 0 ... 152049 =      0.000 ...   608.196 secs
Ready.
2017-10-18 14:50:19,450 INFO : Loading /home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R09-10/AnLaNBD1S001R10_1-1_250Hz.BBCI.mat
Creating RawArray with float64 data, n_channels=64, n_times=151100
    Range : 0 ... 151099 =      0.000 ...   604.396 secs
Ready.
2017-10-18 14:50:20,541 INFO : Preprocessing....
2017-10-18 14:50:20,551 INFO : Just copying data, no resampling, since new sampling rate same.
2017-10-18 14:50:20,711 INFO : Preprocessing....
2017-10-18 14:50:20,720 INFO : Just copying data, no resampling, since new sampling rate same.


## Create the model

We already create the model now, since we need to know the receptive field size for properly cutting out the data to predict. We need to cut out data starting at -receptive_field_size samples before the first sample we want to predict.

In [6]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 650
in_chans = train_cnts[0].get_data().shape[0]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=5, input_time_length=input_time_length,
                        final_conv_length=29).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()
from braindecode.torch_ext.util import np_to_var
import numpy as np
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))
n_receptive_field = input_time_length - n_preds_per_input
receptive_field_ms = n_receptive_field * 1000.0 / train_cnts[0].info['sfreq']
print("Receptive field: {:d}/{:.2f} (samples/ms)".format(n_receptive_field,
                                                      receptive_field_ms))

132 predictions per input/trial
Receptive field: 518/2072.00 (samples/ms)


### Create SignalAndTarget Sets

In [7]:
from braindecode.datautil.trial_segment import create_signal_target_with_breaks_from_mne

break_start_offset_ms = 1000
break_stop_offset_ms = -500

train_sets = [create_signal_target_with_breaks_from_mne(
    cnt, name_to_start_code, [0,0], 
    name_to_stop_code, min_break_length_ms=1000, max_break_length_ms=10000,
    break_epoch_ival_ms=[500,-500],
    prepad_trials_to_n_samples=input_time_length) 
              for cnt in train_cnts]
train_set = concatenate_sets(train_sets)

2017-10-18 14:50:23,917 INFO : Trial per class:
Counter({'Break': 72, 'Right Hand': 29, 'Words': 21, 'Feet': 19, 'Rotation': 4})
2017-10-18 14:50:23,943 INFO : Trial per class:
Counter({'Break': 80, 'Feet': 31, 'Words': 26, 'Right Hand': 18, 'Rotation': 6})
2017-10-18 14:50:23,970 INFO : Trial per class:
Counter({'Break': 95, 'Feet': 38, 'Words': 29, 'Right Hand': 22, 'Rotation': 7})


In [8]:
test_sets = [create_signal_target_with_breaks_from_mne(
    cnt, name_to_start_code, [0,0], 
    name_to_stop_code, min_break_length_ms=1000, max_break_length_ms=10000,
    break_epoch_ival_ms=[500,-500],
    prepad_trials_to_n_samples=input_time_length) 
              for cnt in test_cnts]
test_set = concatenate_sets(test_sets)

2017-10-18 14:50:24,055 INFO : Trial per class:
Counter({'Break': 76, 'Feet': 24, 'Right Hand': 24, 'Words': 19, 'Rotation': 10})
2017-10-18 14:50:24,078 INFO : Trial per class:
Counter({'Break': 80, 'Feet': 30, 'Right Hand': 22, 'Words': 21, 'Rotation': 8})


In [9]:
from braindecode.datautil.splitters import split_into_two_sets

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)


## Setup optimizer and iterator

In [10]:
from torch import optim

optimizer = optim.Adam(model.parameters())

In [11]:
from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

## Setup Monitors, Loss function, Stop Criteria

In [12]:
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression
from braindecode.torch_ext.losses import log_categorical_crossentropy


loss_function = log_categorical_crossentropy

model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

## Run experiment

In [13]:
exp.run()

2017-10-18 14:50:24,318 INFO : Run until first stop...
2017-10-18 14:50:25,203 INFO : Epoch 0
2017-10-18 14:50:25,205 INFO : train_loss                6.69229
2017-10-18 14:50:25,206 INFO : valid_loss                6.46568
2017-10-18 14:50:25,207 INFO : test_loss                 7.03033
2017-10-18 14:50:25,208 INFO : train_sample_misclass     0.82260
2017-10-18 14:50:25,209 INFO : valid_sample_misclass     0.80963
2017-10-18 14:50:25,209 INFO : test_sample_misclass      0.84529
2017-10-18 14:50:25,210 INFO : train_misclass            0.84673
2017-10-18 14:50:25,211 INFO : valid_misclass            0.83838
2017-10-18 14:50:25,212 INFO : test_misclass             0.87261
2017-10-18 14:50:25,213 INFO : runtime                   0.00000
2017-10-18 14:50:25,213 INFO : 
2017-10-18 14:50:25,215 INFO : New best valid_misclass: 0.838384
2017-10-18 14:50:25,216 INFO : 
2017-10-18 14:50:26,150 INFO : Time only for training updates: 0.83s
2017-10-18 14:50:26,968 INFO : Epoch 1
2017-10-18 14:50:26

2017-10-18 14:50:39,512 INFO : test_misclass             0.46497
2017-10-18 14:50:39,513 INFO : runtime                   1.49966
2017-10-18 14:50:39,514 INFO : 
2017-10-18 14:50:40,344 INFO : Time only for training updates: 0.72s
2017-10-18 14:50:41,092 INFO : Epoch 10
2017-10-18 14:50:41,093 INFO : train_loss                0.64813
2017-10-18 14:50:41,094 INFO : valid_loss                1.28606
2017-10-18 14:50:41,095 INFO : test_loss                 1.42089
2017-10-18 14:50:41,096 INFO : train_sample_misclass     0.24833
2017-10-18 14:50:41,097 INFO : valid_sample_misclass     0.44124
2017-10-18 14:50:41,097 INFO : test_sample_misclass      0.47843
2017-10-18 14:50:41,098 INFO : train_misclass            0.24121
2017-10-18 14:50:41,099 INFO : valid_misclass            0.41414
2017-10-18 14:50:41,100 INFO : test_misclass             0.44904
2017-10-18 14:50:41,101 INFO : runtime                   1.72894
2017-10-18 14:50:41,102 INFO : 
2017-10-18 14:50:41,877 INFO : Time only for tr

2017-10-18 14:50:56,005 INFO : Epoch 20
2017-10-18 14:50:56,006 INFO : train_loss                0.50343
2017-10-18 14:50:56,007 INFO : valid_loss                1.28780
2017-10-18 14:50:56,008 INFO : test_loss                 1.54259
2017-10-18 14:50:56,009 INFO : train_sample_misclass     0.17428
2017-10-18 14:50:56,009 INFO : valid_sample_misclass     0.43808
2017-10-18 14:50:56,010 INFO : test_sample_misclass      0.48275
2017-10-18 14:50:56,011 INFO : train_misclass            0.17085
2017-10-18 14:50:56,012 INFO : valid_misclass            0.38384
2017-10-18 14:50:56,012 INFO : test_misclass             0.42994
2017-10-18 14:50:56,013 INFO : runtime                   1.45522
2017-10-18 14:50:56,014 INFO : 
2017-10-18 14:50:56,015 INFO : Setup for second stop...
2017-10-18 14:50:56,018 INFO : Train loss to reach 0.60051
2017-10-18 14:50:56,018 INFO : Run until second stop...
2017-10-18 14:50:56,779 INFO : Epoch 12
2017-10-18 14:50:56,780 INFO : train_loss                0.71576
20

2017-10-18 14:51:12,251 INFO : train_sample_misclass     0.21586
2017-10-18 14:51:12,252 INFO : valid_sample_misclass     0.28866
2017-10-18 14:51:12,253 INFO : test_sample_misclass      0.51124
2017-10-18 14:51:12,254 INFO : train_misclass            0.19316
2017-10-18 14:51:12,254 INFO : valid_misclass            0.27273
2017-10-18 14:51:12,255 INFO : test_misclass             0.48089
2017-10-18 14:51:12,256 INFO : runtime                   1.72200
2017-10-18 14:51:12,257 INFO : 
2017-10-18 14:51:13,203 INFO : Time only for training updates: 0.84s
2017-10-18 14:51:13,953 INFO : Epoch 22
2017-10-18 14:51:13,954 INFO : train_loss                0.65319
2017-10-18 14:51:13,955 INFO : valid_loss                0.82842
2017-10-18 14:51:13,956 INFO : test_loss                 1.39787
2017-10-18 14:51:13,957 INFO : train_sample_misclass     0.25414
2017-10-18 14:51:13,958 INFO : valid_sample_misclass     0.33484
2017-10-18 14:51:13,958 INFO : test_sample_misclass      0.48461
2017-10-18 14:

We arrive at about 54% accuracy. With only 3 sensors and 3 training runs, we cannot get much better :)